<a href="https://colab.research.google.com/github/prawizard/TweetsClassification_NLP/blob/main/TweetEval/Sentiment_Analysis_NLP_Progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [55]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import f1_score
import requests
import collections

In [56]:

import os, string, collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import snowballstemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier

from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model, load_model
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, Dense, Dropout, Convolution1D, MaxPooling1D, SpatialDropout1D, Input 
from keras.layers import GlobalMaxPooling1D, concatenate, LSTM, Bidirectional
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [57]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D 
import numpy as np
from numpy import array
import pandas as pd
from sklearn.model_selection import train_test_split

# Constants

In [58]:
TRAIN_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_text.txt"
TRAIN_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/train_labels.txt"
VAL_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_text.txt"
VAL_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/val_labels.txt"
TEST_TEXT_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_text.txt"
TEST_LABELS_URL="https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/offensive/test_labels.txt"
VOCAB_SIZE=2000

# Access Data from the Files

## Download the .txt files to our runtime

In [59]:
r = requests.get(TRAIN_TEXT_URL, allow_redirects=True)
open('train_text.txt', 'wb').write(r.content)

r = requests.get(TRAIN_LABELS_URL, allow_redirects=True)
open('train_labels.txt', 'wb').write(r.content)

r = requests.get(VAL_TEXT_URL, allow_redirects=True)
open('val_text.txt', 'wb').write(r.content)

r = requests.get(VAL_LABELS_URL, allow_redirects=True)
open('val_labels.txt', 'wb').write(r.content)

r = requests.get(TEST_TEXT_URL, allow_redirects=True)
open('test_text.txt', 'wb').write(r.content)

r = requests.get(TEST_LABELS_URL, allow_redirects=True)
open('test_labels.txt', 'wb').write(r.content)

1720

## Read data from the files downloaded

In [60]:
# Tweets
stream=open("train_text.txt")
tweets=stream.readlines()
stream.close()
val_stream=open("val_text.txt")
val_tweets=val_stream.readlines()
val_stream.close()
test_stream=open("test_text.txt")
test_tweets=test_stream.readlines()
test_stream.close()

# Labels
stream=open("train_labels.txt")
tweetsLabels=stream.readlines()
stream.close()
val_stream=open("val_labels.txt")
val_tweetsLabels=val_stream.readlines()
val_stream.close()
test_stream=open("test_labels.txt")
test_tweetsLabels=test_stream.readlines()
test_stream.close()


# Labels
labels=[0]*len(tweetsLabels)
for i in range(len(tweetsLabels)):
  if tweetsLabels[i].find('\n')!=-1:
    labels[i]=int(re.sub('\n', '', tweetsLabels[i]))
val_labels=[0]*len(val_tweetsLabels)
for i in range(len(val_tweetsLabels)):
  if val_tweetsLabels[i].find('\n')!=-1:
    val_labels[i]=int(re.sub('\n', '', val_tweetsLabels[i]))
test_labels=[0]*len(test_tweetsLabels)
for i in range(len(test_tweetsLabels)):
  if test_tweetsLabels[i].find('\n')!=-1:
    test_labels[i]=int(re.sub('\n', '', test_tweetsLabels[i]))


In [61]:
print('Samples in Training set : ',len(labels),', Validation set : ', len(val_labels),', Test set : ', len(test_labels))

Samples in Training set :  11916 , Validation set :  1324 , Test set :  860


# Pre-processing of the text in tweets

## Remove the '@User' text parts from tweets


In [62]:

for i in range(len(tweets)):
  if tweets[i].find('@user')!=-1:
    tweets[i]=re.sub('@user', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('@user')!=-1:
    val_tweets[i]=re.sub('@user', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('@user')!=-1:
    test_tweets[i]=re.sub('@user', '', test_tweets[i])

## Remove hashtags

In [63]:
for i in range(len(tweets)):
  if tweets[i].find('#[a-zA-Z]+')!=-1:
    tweets[i]=re.sub('#[a-zA-Z]+', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('#[a-zA-Z]+')!=-1:
    val_tweets[i]=re.sub('#[a-zA-Z]+', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('#[a-zA-Z]+')!=-1:
    test_tweets[i]=re.sub('#[a-zA-Z]+', '', test_tweets[i])

## Remove escape characters, unnecessary characters if present and correct wrongly spelt words

In [64]:
for i in range(len(tweets)):
  if tweets[i].find('\n')!=-1:
    tweets[i]=re.sub('\n', '', tweets[i])

for i in range(len(val_tweets)):
  if val_tweets[i].find('\n')!=-1:
    val_tweets[i]=re.sub('\n', '', val_tweets[i])

for i in range(len(test_tweets)):
  if test_tweets[i].find('\n')!=-1:
    test_tweets[i]=re.sub('\n', '', test_tweets[i])

# Unnecessary dots
p='\.\.\.|\.\.'

for i in range(len(tweets)):
  tweets[i]=re.sub(p, '', tweets[i])
  tweets[i]=tweets[i].lower()

for i in range(len(val_tweets)):
  val_tweets[i]=re.sub(p, '', val_tweets[i])
  val_tweets[i]=val_tweets[i].lower()

for i in range(len(test_tweets)):
  test_tweets[i]=re.sub(p, '', test_tweets[i])
  test_tweets[i]=test_tweets[i].lower()

In [65]:
rows=[]
rowIndices=[]
for i in range(len(tweets)):
  rows.append({"TWEET":tweets[i], "CATEGORY":labels[i]})
  rowIndices.append(i+1)
train_df=pd.DataFrame(rows, index=rowIndices)

val_rows=[]
val_rowIndices=[]
for i in range(len(val_tweets)):
  val_rows.append({"TWEET":val_tweets[i], "CATEGORY":val_labels[i]})
  val_rowIndices.append(i+1)
val_df=pd.DataFrame(val_rows, index=val_rowIndices)

test_rows=[]
test_rowIndices=[]
for i in range(len(test_tweets)):
  test_rows.append({"TWEET":test_tweets[i], "CATEGORY":test_labels[i]})
  test_rowIndices.append(i+1)
test_df=pd.DataFrame(test_rows, index=test_rowIndices)

In [66]:
frames=[train_df, val_df, test_df]
df=pd.concat(frames)

In [67]:
type(df)

pandas.core.frame.DataFrame

In [68]:
df

,TWEET,CATEGORY
1,bono who cares. soon people will understand t...,0
2,eight years the republicans denied obama’s pi...,1
3,get him some line help. he is gonna be just f...,0
4,she is great. hi fiona!,0
5,she has become a parody unto herself? she has...,1
...,...,...
856,#cnn irrationally argues 4 legalising #abortio...,0
857,"city of chicago, democrat run wi...",0
858,#conservatives don’t care what you postit’s p...,1
859,#antifa #resist trump is trying to bring world...,0


In [69]:
df['CATEGORY'].value_counts()

0    9460
1    4640
Name: CATEGORY, dtype: int64

In [70]:
text = df['TWEET'].tolist()
text[:10]

[' bono who cares. soon people will understand that they gain nothing from following a phony celebrity. become a leader of your people instead or help and support your fellow countrymen. ',
 ' eight years the republicans denied obama’s picks. breitbarters outrage is as phony as their fake president. ',
 ' get him some line help. he is gonna be just fine. as the game went on you could see him progressing more with his reads. he brought what has been missing. the deep ball presence. now he just needs a little more time ',
 '  she is great. hi fiona! ',
 " she has become a parody unto herself? she has certainly taken some heat for being such an.well idiot. could be optic too  who know with liberals  they're all optics.  no substance ",
 '               this is the vetsresistsquadron"" is bullshit they are girl scout veterans, i have never met any other veterans or served with anyone that was a gun control advocate? have you?"" ',
 ' your looking more like a plant #maga #walkaway ',
 '  lo

In [71]:
y = df['CATEGORY']

In [72]:
token = Tokenizer()
token.fit_on_texts(text)
token

In [73]:
vocab_size = len(token.word_index) + 1
vocab_size

23433

In [74]:
import itertools 
print(dict(itertools.islice(token.index_word.items(), 100)))

{1: 'the', 2: 'is', 3: 'to', 4: 'a', 5: 'and', 6: 'you', 7: 'of', 8: 'are', 9: 'i', 10: 'he', 11: 'in', 12: 'that', 13: 'for', 14: 'she', 15: 'it', 16: 'this', 17: 'on', 18: 'not', 19: 'with', 20: 'they', 21: 'have', 22: 'be', 23: 'liberals', 24: 'gun', 25: 'so', 26: 'all', 27: 'control', 28: 'antifa', 29: 'your', 30: 'like', 31: 'what', 32: 'as', 33: 'but', 34: 'we', 35: 'just', 36: 'about', 37: 'her', 38: 'maga', 39: 'was', 40: 'if', 41: 'conservatives', 42: 'will', 43: 'do', 44: 'who', 45: 'people', 46: 'no', 47: 'his', 48: 'at', 49: 'or', 50: 'my', 51: 'by', 52: 'has', 53: 'from', 54: 'how', 55: 'an', 56: 'out', 57: 'up', 58: 'their', 59: 'me', 60: 'get', 61: 'one', 62: 'amp', 63: 'know', 64: 'trump', 65: 'why', 66: 'when', 67: 'more', 68: 'because', 69: 'can', 70: 'him', 71: 'now', 72: 'them', 73: 'think', 74: "don't", 75: 'there', 76: 'would', 77: 'should', 78: 'right', 79: 'our', 80: 'good', 81: 'only', 82: 'us', 83: "it's", 84: 'want', 85: 'time', 86: 'go', 87: 'see', 88: 'goin

In [75]:
x = ['i to the a and']
token.texts_to_sequences(x)

[[9, 3, 1, 4, 5]]

In [76]:
encoded_text = token.texts_to_sequences(text)
print(encoded_text[:30])

[[1370, 44, 694, 532, 45, 42, 324, 12, 20, 1923, 145, 53, 542, 4, 2279, 3978, 437, 4, 660, 7, 29, 45, 480, 49, 182, 5, 159, 29, 1638, 10016], [3979, 147, 1, 263, 4601, 2875, 3980, 10017, 1566, 2, 32, 2279, 32, 58, 285, 135], [60, 70, 107, 543, 182, 10, 2, 415, 22, 35, 517, 32, 1, 298, 544, 17, 6, 154, 87, 70, 10018, 67, 19, 47, 10019, 10, 1125, 31, 52, 93, 1371, 1, 569, 2021, 3501, 71, 10, 35, 224, 4, 287, 67, 85], [14, 2, 138, 1208, 3981], [14, 52, 437, 4, 6990, 10020, 746, 14, 52, 1372, 850, 107, 5520, 13, 94, 188, 55, 117, 570, 154, 22, 10021, 102, 44, 63, 19, 23, 361, 26, 5521, 46, 6991], [16, 2, 1, 10022, 2, 533, 20, 8, 362, 10023, 2022, 9, 21, 90, 1080, 101, 118, 2022, 49, 2639, 19, 213, 12, 39, 4, 24, 27, 1924, 21, 6], [29, 288, 67, 30, 4, 2876, 38, 352], [161, 586, 260, 1, 119, 1253, 135, 11, 79, 10024, 260, 6992, 119, 7, 1, 2133, 10025, 105, 5, 639, 174, 17, 1, 78, 2431, 158, 38], [93, 4, 3127, 649, 220, 204, 119, 7, 6, 110, 1126, 97, 12, 10, 2, 1081, 4, 1209, 19, 951, 368, 18

In [77]:
ll=len(encoded_text[0])
for tw in encoded_text:
  l=len(tw)
  if l>ll:
    ll=l
ll

62

In [78]:
max_length = 65
X = pad_sequences(encoded_text, maxlen=max_length, padding='post')
print(X)

[[1370   44  694 ...    0    0    0]
 [3979  147    1 ...    0    0    0]
 [  60   70  107 ...    0    0    0]
 ...
 [  41   95  184 ...    0    0    0]
 [  28  888   64 ...    0    0    0]
 [9691    6   89 ...    0    0    0]]


In [79]:
X.shape

(14100, 65)

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2, stratify = y)

In [81]:
vec_size = 350

model = Sequential()
model.add(Embedding(vocab_size, vec_size, input_length=max_length))

model.add(Conv1D(64, 8, activation = 'relu'))
model.add(MaxPooling1D(2))
model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))

model.add(GlobalMaxPooling1D())

model.add(Dense(1, activation='sigmoid'))

In [82]:
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [83]:
%%time
model.fit(X_train, y_train, epochs = 5, validation_data = (X_test, y_test))

Epoch 1/5
353/353 [==============================] - 29s 81ms/step - loss: 0.6421 - accuracy: 0.6529 - val_loss: 0.6070 - val_accuracy: 0.7255
Epoch 2/5
353/353 [==============================] - 28s 80ms/step - loss: 0.4965 - accuracy: 0.7805 - val_loss: 0.5544 - val_accuracy: 0.7379
Epoch 3/5
353/353 [==============================] - 28s 80ms/step - loss: 0.3290 - accuracy: 0.8720 - val_loss: 0.5540 - val_accuracy: 0.7287
Epoch 4/5
353/353 [==============================] - 28s 81ms/step - loss: 0.1964 - accuracy: 0.9311 - val_loss: 0.6252 - val_accuracy: 0.7096
Epoch 5/5
353/353 [==============================] - 28s 80ms/step - loss: 0.1155 - accuracy: 0.9629 - val_loss: 0.6566 - val_accuracy: 0.7319
CPU times: user 4min 3s, sys: 5.87 s, total: 4min 9s
Wall time: 2min 22s


In [84]:
y_pred=(model.predict(X_test)>0.5)*1

In [85]:
y_pred[:5]

array([[0],
       [0],
       [0],
       [0],
       [0]])

In [86]:
y_true=np.array(y_test)
metrics.f1_score(y_true, y_pred, average='macro')


0.6899005608181348

# Naive Bayes

In [87]:
import os, string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import snowballstemmer
from nltk.corpus import stopwords

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin

In [88]:
le = LabelEncoder()
df['target'] = le.fit_transform(df['CATEGORY'])

In [89]:
import re
from sklearn.base import BaseEstimator, TransformerMixin

class TextCleaner(BaseEstimator, TransformerMixin):    
    def remove_mentions(self, text):        
        return re.sub(r'@\w+', '', text)
    
    def remove_urls(self, text):        
        return re.sub(r'http.?://[^\s]+[\s]?', '', text)
    
    def only_characters(self, text):
        return re.sub('[^a-zA-Z\s]', '', text)
    
    def remove_extra_spaces(self, text):
        text = re.sub("\s+", ' ', text)
        text = text.lstrip()
        return text.rstrip()
    
    def to_lower(self, text):
        return text.lower()
    
    def fix_words(self, text):
        text = re.sub(r'\bthx\b', 'thanks', text)
        text = re.sub(r'\bu\b', 'you', text)
        text = re.sub(r'\bhrs\b', 'hours', text)
        text = re.sub(r'\baa\b', 'a', text)
        text = re.sub(r'\bflightr\b', 'flight', text)
        text = re.sub(r'\bur\b', 'your', text)
        text = re.sub(r'\bhr\b', 'hour', text)
        text = re.sub(r'\bthru\b', 'through', text)
        text = re.sub(r'\br\b', 'are', text)
        text = re.sub(r'\bppl\b', 'people', text)
        text = re.sub(r'\btix\b', 'fix', text)
        text = re.sub(r'\bplz\b', 'please', text)
        text = re.sub(r'\bflightd\b', 'flighted', text)
        text = re.sub(r'\btmrw\b', 'tomorrow', text)
        text = re.sub(r'\bthx\b', 'thanks', text)
        text = re.sub(r'\bpls\b', 'please', text)
        text = re.sub(r'\bfyi\b', 'for your information', text)
        
        text = re.sub(r'\bheyyyy\b', 'hey', text)
        text = re.sub(r'\bguyyyys\b', 'guys', text)
        text = re.sub(r'\byall\b', 'you all', text)
        text = re.sub(r'\basap\b', 'as soon as possible', text)
        text = re.sub(r'\bbtw\b', 'by the way', text)
        text = re.sub(r'\bdm\b', 'direct message', text)
        text = re.sub(r'\bcudtomers\b', 'customers', text)
        text = re.sub(r'\bwtf\b', 'what the fuck', text)
        text = re.sub(r'\biphone\b', 'phone', text)
        text = re.sub(r'\bmins\b', 'minutes', text)
        text = re.sub(r'\btv\b', 'television', text)
        text = re.sub(r'\bokay\b', 'ok', text)
        text = re.sub(r'\bfeb\b', 'february', text)
        text = re.sub(r'\byr\b', 'year', text)
        text = re.sub(r'\bshes\b', 'she is', text)
        text = re.sub(r'\bnope\b', 'no', text)
        text = re.sub(r'\bhes\b', 'he is', text)
        text = re.sub(r'\btill\b', 'until', text)
        text = re.sub(r'\bomg\b', 'oh my god', text)
        text = re.sub(r'\btho\b', 'though', text)
        text = re.sub(r'\bnothappy\b', 'not happy', text)
        return re.sub(r'\bthankyou\b', 'thank you', text)
        
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):        
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.only_characters).apply(self.remove_extra_spaces).apply(self.to_lower).apply(self.fix_words)
        return clean_X

In [90]:
ct = TextCleaner()
df['clean_text'] = ct.transform(df['TWEET'])

In [91]:
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
stemmer = snowballstemmer.EnglishStemmer()

def tokenize(s): 
    tokens = re_tok.sub(r' \1 ', s).split()
    return stemmer.stemWords(tokens)

In [92]:
X_train, X_test, y_train, y_test = train_test_split(df['clean_text'].values, df['target'].values, test_size=0.25, random_state=0)

In [93]:
vect = TfidfVectorizer(strip_accents='unicode', tokenizer=tokenize, ngram_range=(1, 2), max_df=0.75, min_df=3, sublinear_tf=True)

In [94]:
tfidf_train = vect.fit_transform(X_train)
tfidf_test = vect.transform(X_test)

In [95]:
z=np.unique(y_train)
z

lk=z[z!=0]
lk
tfidf_train[y_train==0].sum(0)

matrix([[164.4841382 ,   0.96943109,   0.62992181, ...,   0.        ,
           1.21343101,   1.92864503]])

In [96]:
def naive_bayes(x, y):
    r = []; b = []
    labels = np.unique(y)

    for l in labels:
        other_l = labels[labels != l]
        p = x[y == l].sum(0) + 1
        # q = x[(y == other_l[0]) | (y == other_l[1])].sum(0) + 1
        q = x[(y == other_l[0])].sum(0) + 1
        r.append(np.log((p/p.sum())/(q/q.sum())))
        b.append(np.log(len(p)/len(q)))
    
    return r, b

In [97]:
r, b = naive_bayes(tfidf_train, y_train)

pre_preds = []
for j in range(len(r)):
    pre_preds.append(np.asarray(tfidf_test @ r[j].T + b[j]).reshape(-1))
arr = np.array(pre_preds)

In [98]:
np.argmax(arr.T, 1)

array([0, 1, 0, ..., 0, 0, 0])

In [99]:
y_test

array([0, 1, 0, ..., 1, 0, 0])

In [100]:
metrics.accuracy_score(y_test, np.argmax(arr.T, 1))

0.7353191489361702

In [101]:
metrics.f1_score(y_test, np.argmax(arr.T, 1), average='macro')

0.6702360078726444

# Logistic Regression

In [102]:
scores = cross_val_score(LogisticRegression(C=2, dual=True), tfidf_train, y_train, cv=5)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py", line 446, in _check_solver
    "dual=False, got dual=%s" % (solver, dual))
ValueError: Solver lbfgs supports only dual=False, got dual=True

  FitFailedWarning)


In [103]:
scores

array([nan, nan, nan, nan, nan])

In [104]:

m = LogisticRegression()
m.fit(tfidf_train, y_train)

LogisticRegression()

In [105]:
preds = m.predict(tfidf_test)
(preds==y_test).mean()

0.7577304964539007

In [106]:
metrics.f1_score(y_test, preds, average='macro')

0.6629877265668243

# BERT

In [107]:
!pip install ktrain

In [108]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

In [111]:
train_df

,TWEET,CATEGORY
1,bono who cares. soon people will understand t...,0
2,eight years the republicans denied obama’s pi...,1
3,get him some line help. he is gonna be just f...,0
4,she is great. hi fiona!,0
5,she has become a parody unto herself? she has...,1
...,...,...
11912,i wonder if they are sex traffic victims?,1
11913,do we dare say he is better than nyjer?,0
11914,no idea who he is. sorry,0
11915,#professor who shot self over trump says gun c...,0


In [113]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df=train_df,
                                                                   text_column = 'TWEET',
                                                                   label_columns = 'CATEGORY',
                                                                   val_df = test_df,
                                                                   maxlen = 65,
                                                                   preprocess_mode = 'bert')

['not_CATEGORY', 'CATEGORY']
   not_CATEGORY  CATEGORY
1           1.0       0.0
2           0.0       1.0
3           1.0       0.0
4           1.0       0.0
5           0.0       1.0
['not_CATEGORY', 'CATEGORY']
   not_CATEGORY  CATEGORY
1           0.0       1.0
2           1.0       0.0
3           1.0       0.0
4           1.0       0.0
5           1.0       0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [114]:
model = text.text_classifier(name = 'bert',
                             train_data = (X_train, y_train),
                             preproc = preproc)

Is Multi-Label? False
maxlen is 65
done.


In [115]:
learner = ktrain.get_learner(model=model, train_data=(X_train, y_train),
                   val_data = (X_test, y_test),
                   batch_size = 6)

In [116]:
learner.fit_onecycle(lr = 2e-5, epochs = 1)

predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/My Drive/bert')



begin training using onecycle policy with max lr of 2e-05...
1986/1986 [==============================] - 596s 293ms/step - loss: 0.5492 - accuracy: 0.7196 - val_loss: 0.3599 - val_accuracy: 0.8628


In [119]:
data = test_df['TWEET'].tolist()

In [127]:
bert_pred=predictor.predict(data)

In [133]:
bert_pred[5:10]

['not_CATEGORY', 'not_CATEGORY', 'not_CATEGORY', 'CATEGORY', 'CATEGORY']

In [143]:
y_true=np.array(test_df['CATEGORY'].tolist())

In [137]:
res_bert=[]
for i in range(len(bert_pred)):
  if bert_pred[i]=='CATEGORY':
    res_bert.append(1)
  else:
    res_bert.append(0)

In [144]:
res_bert=np.array(res_bert)

In [146]:
np.mean(res_bert==y_true)

0.8627906976744186

In [147]:
metrics.f1_score(y_true, res_bert, average='macro')

0.8263411161536303